In [19]:
import os
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

In [20]:
import torch
import torch.nn as nn
import torch.functional as F

In [21]:
from typical import cf_user_based
import utils

In [22]:
from sklearn.model_selection import train_test_split

fpath = os.path.join(os.getcwd(), "DoubanBook", "user_book.dat")
# df = utils.read_file(fpath)
# train, test = train_test_split(df, test_size=0.2)
# train, valid = train_test_split(train, test_size=0.1)

# train.to_csv(os.path.join(os.getcwd(), "DoubanBook", "train.dat"), sep = "\t", index=False, header=False)
# test.to_csv(os.path.join(os.getcwd(), "DoubanBook", "test.dat"), sep = "\t", index=False, header=False)
# valid.to_csv(os.path.join(os.getcwd(), "DoubanBook", "valid.dat"), sep = "\t", index=False, header=False)

In [23]:
root = os.path.join(os.getcwd(), "DoubanBook")
user = 13024
item = 22347

In [24]:
trainp = os.path.join(root, "train.dat")
validp = os.path.join(root, "valid.dat")
tesp = os.path.join(root, "test.dat")

In [25]:
train_df = utils.read_file(trainp)
train_interact = csr_matrix((train_df.cols_2, (train_df.cols_0, train_df.cols_1)), shape=(user+1, item+1))
train_interact = train_interact.toarray()

valid_df = utils.read_file(validp)
# train_interact = csr_matrix((train_df.cols_2, (train_df.cols_0, train_df.cols_1)), shape=(user+1, item+1))

test_df = utils.read_file(tesp)
# train_interact = csr_matrix((train_df.cols_2, (train_df.cols_0, train_df.cols_1)), shape=(user+1, item+1))

In [7]:
scores = cf_user_based(1153, train_interact)

In [17]:
ten = torch.tensor(scores[1:])
ten.size()

torch.Size([13024])

In [18]:
torch.nn.Sigmoid()(ten)

tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)

In [8]:
a = np.array([1, 2, 3]).reshape(1, -1)
b = np.array([3, 9, 2]).reshape(1, -1)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
np.squeeze(cosine_similarity(a, b))[0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [41]:
df.cols_0

0         10855
1         10027
2           741
3           453
4         11665
          ...  
792057    12832
792058     7823
792059     9347
792060    10942
792061     5165
Name: cols_0, Length: 792062, dtype: int64

In [62]:
M = csr_matrix((df.cols_2, (df.cols_0, df.cols_1)), shape=(13024, 22347))
M

ValueError: row index exceeds matrix dimensions

In [50]:
arr = M.toarray()

In [57]:
tmp = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])

In [61]:
np.delete(tmp, 1, 1)

array([[ 1,  3,  4],
       [ 5,  7,  8],
       [ 9, 11, 12]])

In [37]:
np.loadtxt(fpath, dtype=int)

array([[10855,   938,     4],
       [10027,     3,     3],
       [  741,  2426,     5],
       ...,
       [ 9347, 18017,     5],
       [10942,  1443,     4],
       [ 5165,   772,     3]])

In [42]:
pd.read_csv(fpath, sep="\t", header=None).columns

Int64Index([0, 1, 2], dtype='int64')

In [43]:
a = [[1, 2], [3, 4]]
a = np.array(a)

In [44]:
a[:, 0]

array([1, 3])

In [45]:
fpath_1 = os.path.join(os.getcwd(), "DoubanBook", "user_group.dat")
fpath_2 = os.path.join(os.getcwd(), "DoubanBook", "user_location.dat")

In [46]:
df1 = utils.read_file(fpath_1)

In [47]:
len(pd.unique(df1['cols_0']))

13024

In [48]:
max(pd.unique(df1['cols_0']))

13024

In [49]:
min(pd.unique(df1['cols_0']))

1

In [17]:
len(pd.unique(df1['cols_1']))

2936

In [19]:
max(pd.unique(df1['cols_1']))

2936

In [10]:
df2 = utils.read_file(fpath_2)
df

,cols_0,cols_1
0,3587,33
1,3210,179
2,7993,394
3,6674,204
4,12434,33
...,...,...
10587,5823,261
10588,4860,33
10589,9107,421
10590,6025,162


In [11]:
utils.get_explicit_features(df1, df2)

,cols_0,cols_1_x,cols_1_y
0,3587,232,33.0
1,3587,666,33.0
2,3587,226,33.0
3,3587,108,33.0
4,3587,63,33.0
...,...,...,...
1189266,8753,400,68.0
1189267,8753,14,68.0
1189268,8753,2882,68.0
1189269,8753,2556,68.0
